In [23]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
url_list = pd.read_csv('./data/series_urls.csv')
from urllib.parse import urljoin

In [ ]:
base_url = 'https://en.wikipedia.org/'

for index, row in url_list.iterrows():
    series = row['Series']
    print(series)
    soup = BeautifulSoup(urlopen(row['URL']))
    cast_table = soup.find('table', class_='wikitable')
    rows = cast_table.find_all('tr')

    # Each table row contains the housewife's name and the seasons they were active in.
    for row in rows[2:]:
        # Ignore Friends of the Housewives since they're not the main cast.
        if row.text.strip().lower() == 'friends of the housewives':
            break

        # The table data in the row contains the housewife's name and their role in a particular season.
        td = row.find('td')

        birthdate = None # Reset the birthdate each loop, otherwise wrong birthdate may be carried over.
        housewife_url = td.find('a')
        if housewife_url != None:
            housewife_url = urljoin(base_url, housewife_url.get("href"))
            req = requests.get(housewife_url)
            html = BeautifulSoup(req.text, 'html.parser')
            biography_table = html.find('table', class_='infobox')
            birthdate = biography_table.find('span', class_='bday')

            if birthdate != None:
                birthdate = birthdate.text

        name = td.text.strip()
        name_parts = name.split(' ', 1)
        print(f"""First: {name_parts[0]}
Last: {name_parts[1]}
Birthdate: {birthdate}
""")